## Imports

In [39]:
import pandas as pd
import numpy as np

## Load `train.csv`

In [40]:
df: pd.DataFrame = pd.read_csv('train.csv')

In [41]:
df.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [42]:
df[df["MISSING_DATA"] == True].index

Int64Index([ 105621,  171397,  299137,  457486,  738466,  782321,  848552,
             932391, 1275934, 1432196],
           dtype='int64')

## Delete rows where `MISSING_DATA` is true and delete column `MISSING_DATA`

In [43]:
df.drop(df[df["MISSING_DATA"] == True].index, inplace=True)

In [44]:
df.drop(labels="MISSING_DATA", axis=1, inplace=True)

## Delete column `DAY_TYPE`

In [45]:
df.drop(labels="DAY_TYPE", axis=1, inplace=True)

## Delete column `TRIP_ID`

In [46]:
df.drop(labels="TRIP_ID", axis=1, inplace=True)

## Set `ORIGIN_STAND` to 0 for entries with null `ORIGIN_STAND`

In [47]:
df.at[pd.isna(df["ORIGIN_STAND"]) == True, "ORIGIN_STAND"] = 0

## Set `ORIGIN_CALL` to 0 for entries with null `ORIGIN_CALL`

In [50]:
df.at[pd.isna(df["ORIGIN_CALL"]) == True, "ORIGIN_CALL"] = 0

## Assigning integers to `CALL_TYPE`

In [52]:
df.at[df["CALL_TYPE"] == "A", "CALL_TYPE"] = 0
df.at[df["CALL_TYPE"] == "B", "CALL_TYPE"] = 1
df.at[df["CALL_TYPE"] == "C", "CALL_TYPE"] = 2
df["CALL_TYPE"] = df["CALL_TYPE"].astype(np.int32)

## Convert `POLYLINE` column to `TARGET` column.

In [68]:
df["TARGET"] = df["POLYLINE"].apply(lambda x : 15 * max(x.count("[") - 1, 0))
df.drop(labels="POLYLINE", axis=1, inplace=True)

In [69]:
df.head()

,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,TARGET
0,2,0.0,0.0,20000589,1372636858,345
1,1,0.0,7.0,20000596,1372637303,285
2,2,0.0,0.0,20000320,1372636951,975
3,2,0.0,0.0,20000520,1372636854,645
4,2,0.0,0.0,20000337,1372637091,435


In [ ]:
# TODO: convert timestamp column to date + time + day of week n stuff